In [4]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files='sample2k.csv')
dataset = dataset['train'].train_test_split(test_size=0.1)


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "Qwen/Qwen1.5-1.8B-Chat"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    device_map=None  # or remove this line entirely
)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
C:\Users\kbdha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kbdha\.cache\huggingface\hub\models--Qwen--Qwen1.5-1.8B-Chat. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or t

In [6]:
def format_conversation(example):
    prompt = example['instruction'].strip()
    response = example['output'].strip()
    full_text = f"User: {prompt}\nBot: {response}"
    
    # Tokenize
    encoding = tokenizer(
        full_text,
        truncation=True,
        padding="max_length",
        max_length=512
    )

    input_ids = encoding["input_ids"]
    attention_mask = encoding["attention_mask"]

    # Create labels (same as input_ids, mask padding tokens with -100)
    labels = input_ids.copy()
    labels = [label if label != tokenizer.pad_token_id else -100 for label in labels]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

# Re-tokenize the dataset
tokenized_dataset = dataset.map(format_conversation)


Map: 100%|██████████| 200/200 [00:00<00:00, 295.86 examples/s]


In [7]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig
import torch

# Load the base model and force it to CPU manually after loading
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen1.5-1.8B-Chat",
    trust_remote_code=True,         # Qwen requires this
    torch_dtype=torch.float32       # FP32 is safe for CPU
)

model = model.to("cpu")  # Move to CPU after loading

# Apply LoRA configuration (Low-Rank Adaptation)
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Inject LoRA into model
model = get_peft_model(model, peft_config)


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./qwen-lora-safety",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    save_steps=100,
    save_total_limit=2,
    logging_steps=10,
    learning_rate=2e-4,
    bf16=False,  # if supported
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"]
)

trainer.train()
model.save_pretrained("qwen-lora-safety-model")
tokenizer.save_pretrained("qwen-lora-safety-model")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
Exception ignored in: <function tqdm.__del__ at 0x000002033DCCE700>
Traceback (most recent call last):
  File "C:\Users\kbdha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "C:\Users\kbdha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\std.py", line 1265, in close
    def close(self):

KeyboardInterrupt: 
C:\Users\kbdha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set